In [1]:
import os
import time
from math import floor, ceil

import numpy
from keras import layers, metrics
from keras import models, optimizers
from keras.callbacks import ModelCheckpoint
from keras.applications import VGG16
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from utils import *
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

start = time.time()

train_dir = '/home/snowflake/Documents/nowe_trendy/train'
test_dir = '/home/snowflake/Documents/nowe_trendy/test'
validation_dir = '/home/snowflake/Documents/nowe_trendy/validation'

path, dirs, files = os.walk(train_dir).__next__()

val_size = 0.15
nTrain = sum([len(files) for r, d, files in os.walk(train_dir)]) - len(dirs)
nVal = ceil(nTrain * val_size)
path, dirs, files = os.walk(test_dir).__next__()
nTest = sum([len(files) for r, d, files in os.walk(test_dir)]) - len(dirs)
nClasses = len(dirs)
batch_size = 20
dense_size = 256
epochs = 10

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

for layer in conv_base.layers[:-4]:
    layer.trainable = False
    
for layer in conv_base.layers:
    print(layer, layer.trainable)

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(dense_size, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(nClasses, activation='softmax'))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<keras.engine.topology.InputLayer object at 0x7fb7e843d470> False
<keras.layers.convolutional.Conv2D object at 0x7fb767676748> False
<keras.layers.convolutional.Conv2D object at 0x7fb767676550> False
<keras.layers.pooling.MaxPooling2D object at 0x7fb767676828> False
<keras.layers.convolutional.Conv2D object at 0x7fb767628898> False
<keras.layers.convolutional.Conv2D object at 0x7fb767628eb8> False
<keras.layers.pooling.MaxPooling2D object at 0x7fb76764e208> False
<keras.layers.convolutional.Conv2D object at 0x7fb7673ca320> False
<keras.layers.convolutional.Conv2D object at 0x7fb7673d9dd8> False
<keras.layers.convolutional.Conv2D object at 0x7fb7673fde10> False
<keras.layers.pooling.MaxPooling2D object at 0x7fb767390780> False
<keras.layers.convolutional.Conv2D object at 0x7fb7673a2588> False
<keras.layers.convolutional.Conv2D object at 0x7fb767348d68> False
<keras.layers.convolutional.Conv2D object at 0x7fb769521358> False
<keras.layers.pooling.MaxPooling2D object at 0x7fb76736b5f8> Fa

In [2]:
epochs = 20
train_batchsize = 12
val_batchsize = 8
dense_size = 256

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   fill_mode="nearest",
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 7875 images belonging to 12 classes.
Found 1818 images belonging to 12 classes.


In [4]:
print("**********Configure model for training************")

mcp_save = ModelCheckpoint('best_model_acc.model', save_best_only=True, monitor='val_categorical_accuracy', mode='max')
model.compile(optimizer=optimizers.Adamax(),
              loss='categorical_crossentropy',
              metrics=[metrics.categorical_accuracy])

**********Configure model for training************


In [ ]:
print("**********Start of training************")
history = model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples/train_generator.batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples/validation_generator.batch_size,
                    callbacks = [mcp_save])

**********Start of training************
Epoch 1/20
657/656 [==============================] - 2414s 4s/step - loss: 2.4549 - categorical_accuracy: 0.1433 - val_loss: 2.4515 - val_categorical_accuracy: 0.1315
Epoch 2/20
657/656 [==============================] - 2591s 4s/step - loss: 2.4206 - categorical_accuracy: 0.1487 - val_loss: 2.4539 - val_categorical_accuracy: 0.1315
Epoch 3/20
657/656 [==============================] - 2590s 4s/step - loss: 2.4216 - categorical_accuracy: 0.1483 - val_loss: 2.4520 - val_categorical_accuracy: 0.1315
Epoch 4/20
657/656 [==============================] - 2594s 4s/step - loss: 2.4198 - categorical_accuracy: 0.1488 - val_loss: 2.4556 - val_categorical_accuracy: 0.1315
Epoch 5/20
657/656 [==============================] - 2651s 4s/step - loss: 2.4201 - categorical_accuracy: 0.1485 - val_loss: 2.4540 - val_categorical_accuracy: 0.1315
Epoch 6/20
657/656 [==============================] - 2575s 4s/step - loss: 2.4203 - categorical_accuracy: 0.1489 - val_

In [ ]:
model.save('last_model_acc.model')
from keras.models import load_model
model = load_model('best_model_acc.model')

In [ ]:
# Create a generator for prediction
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = '/home/snowflake/Documents/nowe_trendy/test'
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = test_generator.filenames
 
# Get the ground truth from generator
ground_truth = test_generator.classes
 
# Get the label to class mapping from the generator
label2index = test_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)

In [ ]:
errors = np.where(predicted_classes.round() != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),test_generator.samples))
accuracy = accuracy_score(ground_truth, predicted_classes)
print("Accuracy: {}".format(accuracy))

In [ ]:
# print("**********End of training************\n\n")

# ground_truth = [np.where(r == 1)[0][0] for r in test_labels]

# predictions = model.predict_classes(test_features)
# prob = model.predict(test_features)

# # errors = nVal - accuracy_score(ground_truth, predictions, normalize=False)
# errors = np.where(predictions != ground_truth)[0]
# accuracy = accuracy_score(ground_truth, predictions)
# precision, recall, f_score, support = score(ground_truth, predictions, average='weighted')

# print("No of errors = {}/{}".format(len(errors), nTest))
# print("Accuracy: {}".format(accuracy))
# print("Weighted average Precision: {}".format(precision))
# print("Weighted average Recall: {}".format(recall))
# print("Weighted average F-score: {}".format(f_score))

# end = time.time()
# print("Time: " + str(end - start))
